# Welcome to Xilinx Maximal Independent Set Acceleration Demo
---
This notebook demonstrates how to use the Xilinx Maximal Independent Set (MIS) product and shows the power of Xilinx Alveo FPGAs cards to accelerate MIS.

### The demo
In this demo, we will use the Xilinx MIS python module (**xilMisPython**) to find MIS on a small synthetic graph stored in Compressed Sparse Row (CSR) format. The graph CSR data is stored in separate row pointer and column index files which are transfered to the Alveo device. The computed MIS on that data is returned as a list of vertex indices.

>**NOTE**: Xilinx MIS module requires atleast ***python 3.6***

### Setup
---
Let's start by importing the module and setting up demo variables

In [ ]:
import xilMisPython as xmis
import os
import struct
import time

We create a function to read and parse the binary data that CSR graph is stored in:

In [ ]:
def readBin(filename, readSize):
    with open(filename, 'rb') as fh:
        fileSize = os.path.getsize(filename)

        if 0 < readSize != fileSize:
            print(f"WARNING: file {filename} size {fileSize} doesn't match required size {readSize}")

        assert (fileSize >= readSize)

        vec = []
        for i in range(int(fileSize/4)):
            vec.append(struct.unpack('<i', fh.read(4))[0])  # read bytes as little-endian 4 byte integers

    return vec

The run is setup with the following options:
- xclbin_path: full path to the Alveo FPGA device executable
- deviceNames: full name of the Alveo board to run on (Supported Alveo U50 and U55C)
- in_dir: location of the data files

The variables here are set from the environment variables in the *run* script used to launch the jupyter server.

In [ ]:
xclbin_path = os.environ.get('XCLBIN_FILE')
deviceNames = os.environ.get('DEV_NAME')
in_dir = os.environ.get('DATA_DIR')

In [ ]:
# set options for MIS
opt = xmis.options()
opt.xclbinPath = xmis.xString(xclbin_path)
opt.deviceNames = xmis.xString(deviceNames)

In [ ]:
# read the input file matrix meta data information stored in infos.txt
with open(in_dir + "/infos.txt") as fh:
    fh.readline()
    n = int(fh.readline())
    fh.readline()
    nz = int(fh.readline())

### Load Graph and initialize compute
---
We use the meta data to load graph data as CSR arrays and create a CSR graph object that is part of the Xilinx MIS library

In [ ]:
# create CSR arrays and graph object
h_rowPtr = readBin(in_dir + "/rowPtr.bin", (n + 1) * 4)  # multiplier of 4 for integer data type
h_colIdx = readBin(in_dir + "/colIdx.bin", nz * 4)       # multiplier of 4 for integer data type

graph = xmis.GraphCSR(h_rowPtr, h_colIdx)

Now let's create the Xilinx MIS object using the options and initialize it with the graph that we just created

In [ ]:
# create MIS object
mis = xmis.MIS(opt)

# initialize MIS object with the graph
mis.setGraph(graph)

Next, the Alveo card needs to be prepared for the MIS run. This step involves establishing connection with the Alveo device and program the FPGA with executable binary called xclbin (if not already done).

In [ ]:
# initialize the FPGA device for MIS run
mis.startMis()

### Execute  MIS
---
Finally, the computation is executed by calling the following API. This transfers (DMA) data to the Alveo HBMs and runs the Xilinx Maximal Indepdent Set algorithm to finid a large size maximal set. Counters are used here to time the run which shows the incredibly fast return time of the Xilinx MIS librrary.

In [ ]:
# execute MIS on FPGA
start = time.perf_counter()

mis_vertex_list = mis.executeMIS()

end = time.perf_counter()
elapsed = end-start

To check the quality of result, the size of the MIS can be found by:

In [ ]:
# get size of the MIS
vertex_count = mis.count()  # can also do len(mis_vertex_list)

print(f"\nFound MIS with {vertex_count} vertices within {elapsed:0.6f} sec")